# Import the package and dataset

In [99]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, Normalizer,MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
from keras.models import load_model
from keras.optimizers import Adam

In [7]:
data=pd.read_csv("train.csv")

In [8]:
test=pd.read_csv("test.csv")

In [9]:
set(test['Accident_Type_Code'])

{1, 2, 3, 4, 5, 6, 7}

In [10]:
data.head()

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054


In [11]:
data.isnull().sum()

Severity                   0
Safety_Score               0
Days_Since_Inspection      0
Total_Safety_Complaints    0
Control_Metric             0
Turbulence_In_gforces      0
Cabin_Temperature          0
Accident_Type_Code         0
Max_Elevation              0
Violations                 0
Adverse_Weather_Metric     0
Accident_ID                0
dtype: int64

# Study the correlation of independent and dependent variable

In [12]:
cor_m=data.ix[:,0:11].corr()

C:\Users\karth\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
C:\Users\karth\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


In [13]:
cor_col=[]
alpha=0.05
for i in range(1,12):
    _,p=spearmanr(data.iloc[:,0],data.iloc[:,i])
    if p<0.05:
        cor_col.append(data.columns[i])

In [14]:
len(data.columns)

12

In [15]:
x=data[cor_col]

# Data analysis

In [16]:
x=data.ix[:,1:11]
y=data.ix[:,0]

C:\Users\karth\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
C:\Users\karth\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  


# One hot encoding of Accident_Type_Code

In [17]:
o_h=pd.get_dummies(data['Accident_Type_Code'],drop_first=True)

In [18]:
del x['Accident_Type_Code']

In [19]:
x=pd.concat([x,o_h],axis=1)

# Train and test split

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

# XGBoost model

In [21]:
from xgboost import XGBClassifier

In [22]:
xg_model=XGBClassifier().fit(x.values,y)

In [23]:
target=xg_model.predict(x_test.values)

# RF model

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
rf_model=RandomForestClassifier().fit(x.values,y)

C:\Users\karth\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [26]:
target=rf_model.predict(x_test.values)

# Extra Tree classifier

In [27]:
from sklearn.ensemble import ExtraTreesClassifier

In [28]:
et_model=ExtraTreesClassifier().fit(x_train.values,y_train)

C:\Users\karth\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [29]:
target=et_model.predict(x_test.values)

# Adaboost

In [30]:
from sklearn.ensemble import AdaBoostClassifier

In [31]:
from sklearn.tree import DecisionTreeClassifier

In [32]:
ab_model=AdaBoostClassifier().fit(x.values,y)

In [33]:
dt_model=DecisionTreeClassifier().fit(x.values,y)

# SVM

In [34]:
from sklearn.svm import SVC

In [35]:
sv_model=SVC(kernel='rbf',C=100,gamma=0.6)

In [36]:
sv_model.fit(x.values,y)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.6, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [37]:
sv_model.score(x_test,y_test)

0.3235

In [38]:
sv_model.score(x_train,y_train)

1.0

# K nn 

In [51]:
from sklearn.neighbors import KNeighborsClassifier

In [52]:
Kn_model=KNeighborsClassifier().fit(x,y)

# Voting classifier

In [619]:
from sklearn.ensemble import VotingClassifier

In [623]:
eclf1 = VotingClassifier(estimators=[('xg', xg_model), ('rf', rf_model), ('et', et_model)], voting='soft')

In [624]:
eclf1 = eclf1.fit(x, y)

In [537]:
eclf1.predict(x_test)

array(['Minor_Damage_And_Injuries', 'Highly_Fatal_And_Damaging',
       'Significant_Damage_And_Serious_Injuries', ...,
       'Highly_Fatal_And_Damaging', 'Highly_Fatal_And_Damaging',
       'Significant_Damage_And_Fatalities'], dtype=object)

In [523]:
x.shape

(10000, 15)

# Stacking classifier

In [39]:
from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression

In [62]:
from sklearn.feature_selection import SelectFromModel

In [68]:
s_l=SelectFromModel(LogisticRegression(penalty='l2',C=1)).fit(x,y)

C:\Users\karth\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\karth\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [41]:
lr_model=LogisticRegression().fit(x,y)

C:\Users\karth\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\karth\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [69]:
s_model=StackingClassifier(classifiers=[xg_model,rf_model,et_model,ab_model,dt_model],use_probas=True,meta_classifier=s_l)

In [70]:
s_model.fit(x.values,y)

C:\Users\karth\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\karth\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


StackingClassifier(average_probas=False,
                   classifiers=[XGBClassifier(base_score=0.5, booster='gbtree',
                                              colsample_bylevel=1,
                                              colsample_bynode=1,
                                              colsample_bytree=1, gamma=0,
                                              learning_rate=0.1,
                                              max_delta_step=0, max_depth=3,
                                              min_child_weight=1, missing=None,
                                              n_estimators=100, n_jobs=1,
                                              nthread=None,
                                              objective='multi:softprob',
                                              random_state=0, reg_alpha=0,
                                              reg_lambda=1,...
                                                                                dual=False,
           

In [702]:
target=s_model.predict(x_test.values)

# Regularization

# Deep learning model

In [79]:
x_r=x.iloc[:,0:9]
col=x_r.columns
encoder=MinMaxScaler().fit(x_r)
x_r=encoder.transform(x_r)
x_r=pd.DataFrame(x_r)
x=x.drop(col,axis=1)
x=pd.concat([x_r,x],axis=1)

In [80]:
y=pd.get_dummies(y)
y_dic={i:j for i,j in enumerate(y.columns)}
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [120]:
model=Sequential()
model.add(Dense(200, input_shape=(x_train.shape[1],)))
model.add(Dense(200))
model.add(Dropout(0.2))
model.add(Dense(200))
model.add(Dense(200))
model.add(Dense(y.shape[1]))

In [127]:
opt=Adam(0.0001)

In [128]:
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['acc'])

In [129]:
model.fit(x,y,epochs=30)

Epoch 1/30
10000/10000 [==============================] - 2s 173us/step - loss: 7.1081 - acc: 0.3693
Epoch 2/30
10000/10000 [==============================] - 1s 148us/step - loss: 7.1145 - acc: 0.3699
Epoch 3/30
10000/10000 [==============================] - 1s 147us/step - loss: 7.0887 - acc: 0.3670
Epoch 4/30
10000/10000 [==============================] - 1s 146us/step - loss: 7.0694 - acc: 0.3681
Epoch 5/30
10000/10000 [==============================] - 1s 149us/step - loss: 7.1065 - acc: 0.3676
Epoch 6/30
10000/10000 [==============================] - 1s 148us/step - loss: 7.1000 - acc: 0.3675
Epoch 7/30
10000/10000 [==============================] - 1s 148us/step - loss: 7.0710 - acc: 0.3697
Epoch 8/30
10000/10000 [==============================] - 1s 147us/step - loss: 7.0694 - acc: 0.3699
Epoch 9/30
10000/10000 [==============================] - 1s 147us/step - loss: 7.0920 - acc: 0.3683
Epoch 10/30
10000/10000 [==============================] - 1s 145us/step - loss: 7.0920 - a

In [342]:
model.save("deep.h5")

In [91]:
model=load_model("deep.h5")

In [348]:
target=model.predict_classes(x_test)

In [107]:
target=[y_dic[i] for i in target]

KeyError: 'Highly_Fatal_And_Damaging'

In [352]:
y_test

8889                  Minor_Damage_And_Injuries
10                    Highly_Fatal_And_Damaging
8650    Significant_Damage_And_Serious_Injuries
7253                  Highly_Fatal_And_Damaging
4963                  Highly_Fatal_And_Damaging
                         ...                   
2761                  Highly_Fatal_And_Damaging
9545                  Minor_Damage_And_Injuries
4247                  Highly_Fatal_And_Damaging
4656          Significant_Damage_And_Fatalities
3158    Significant_Damage_And_Serious_Injuries
Length: 2000, dtype: object

# Deep Test submission

In [336]:
test_x=test.ix[:,0:10]
o_h=pd.get_dummies(test_x['Accident_Type_Code'],drop_first=True)
del test_x['Accident_Type_Code']
test_x=pd.concat([test_x,o_h],axis=1)

C:\Users\karth\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
C:\Users\karth\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


In [338]:
test_en=test_x.iloc[:,0:9]

encoder=MinMaxScaler().fit(test_en)
test_en=encoder.transform(test_en)
test_en=pd.DataFrame(test_en)


In [339]:
test_x=test_x.drop(col,axis=1)
test_x=pd.concat([test_en,test_x],axis=1)

In [354]:
target=model.predict_classes(test_x)

In [355]:
target=[y_dic[i] for i in target]

In [357]:
submission=pd.DataFrame({'Accident_ID':list(test['Accident_ID']),'Severity':list(target)})

In [358]:
submission.to_csv("deep_submission.csv",index=False)

# Accuracy

In [682]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score

In [703]:
accuracy_score(y_test,target)

0.9255

In [704]:
confusion_matrix(y_test,target)

array([[557,  11,  12,  15],
       [ 13, 497,   8,  15],
       [ 12,   7, 296,   6],
       [ 13,  28,   9, 501]], dtype=int64)

In [705]:
f1_score(y_test,target,average='weighted')

0.9254974573656012

In [ ]:
# 84 - 76.23


In [ ]:
#93.55

# Test Dataset

In [44]:
test

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,19.497717,16,6,72.151322,0.388959,78.32,4,37949.724386,2,0.069692,1
1,58.173516,15,3,64.585232,0.250841,78.60,7,30194.805567,2,0.002777,10
2,33.287671,15,3,64.721969,0.336669,86.96,6,17572.925484,1,0.004316,14
3,3.287671,21,5,66.362808,0.421775,80.86,3,40209.186341,2,0.199990,17
4,10.867580,18,2,56.107566,0.313228,79.22,2,35495.525408,2,0.483696,21
...,...,...,...,...,...,...,...,...,...,...,...
2495,68.127854,8,11,49.680948,0.470819,77.80,2,48251.586622,2,0.659387,12484
2496,54.840183,9,9,71.194166,0.417087,79.77,6,20761.984416,1,0.005010,12487
2497,36.712329,14,10,91.203282,0.313950,83.96,6,36676.100601,2,0.008629,12488
2498,38.721461,13,14,73.154057,0.275003,77.38,6,24211.359510,1,0.006247,12491


In [71]:
test_x=test.iloc[:,0:10]

In [72]:
o_h=pd.get_dummies(test_x['Accident_Type_Code'],drop_first=True)
del test_x['Accident_Type_Code']


In [73]:
test_x=pd.concat([test_x,o_h],axis=1)

In [74]:
test_x.shape

(2500, 15)

In [78]:
s_l.get_support()

array([ True,  True, False, False, False,  True, False, False, False,
       False,  True,  True, False, False, False])

In [76]:
target=s_model.predict(test_x.values)

AttributeError: 'SelectFromModel' object has no attribute 'predict'

In [ ]:
submission=pd.DataFrame({'Accident_ID':list(test['Accident_ID']),'Severity':list(target)})

In [ ]:
submission.to_csv("submission.csv",index=False)